In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the data
df = pd.read_excel("CustomerChurnDataset.xlsx", engine='openpyxl')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,numAdminTickets,numTechTickets,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,0,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,One year,No,Mailed check,56.95,1889.5,0,0,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,0,0,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0,3,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,0,0,Yes


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


### Check where missing values occur

In [4]:
df.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
numAdminTickets     0
numTechTickets      0
Churn               0
dtype: int64

No NULL values found

### Clean categorical data
Check unique values for consistency

In [5]:
catogerical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

In [6]:
catogerical_cols

['customerID',
 'gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'TotalCharges',
 'Churn']

In [7]:
for col in catogerical_cols:
    if col != 'customerID':
        print(f"{col}: {df[col].unique()}")

gender: ['Female' 'Male']
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
PhoneService: ['No' 'Yes']
MultipleLines: ['No phone service' 'No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes' 'No internet service']
OnlineBackup: ['Yes' 'No' 'No internet service']
DeviceProtection: ['No' 'Yes' 'No internet service']
TechSupport: ['No' 'Yes' 'No internet service']
StreamingTV: ['No' 'Yes' 'No internet service']
StreamingMovies: ['No' 'Yes' 'No internet service']
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
TotalCharges: [29.85 1889.5 108.15 ... 346.45 306.6 6844.5]
Churn: ['No' 'Yes']


In [8]:
df['TotalCharges'].nunique()

6531

In [9]:
# Convert 'TotalCharges' to numeric
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
print("NaN values in TotalCharges: ", df['TotalCharges'].isnull().sum())
print("TotalCharges Dtype", df['TotalCharges'].dtype)

NaN values in TotalCharges:  11
TotalCharges Dtype float64


As we already have MonthlyCharges and tenure data, we can fill the missing values using the formula:

$TotalCharges = MonthlyCharges × tenure$

In [10]:
df['TotalCharges'] = df['TotalCharges'].fillna(df['MonthlyCharges'] * df['tenure'])
print("NaN values in TotalCharges: ", df['TotalCharges'].isnull().sum())

NaN values in TotalCharges:  0


### Clean categorical columns for Power BI

In [11]:
replace_cols = ['MultipleLines', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

for col in replace_cols:
    df[col] = df[col].replace({'No internet service': 'No', 'No phone service': 'No'})
    print(f"{col}: {df[col].unique()}")

MultipleLines: ['No' 'Yes']
OnlineSecurity: ['No' 'Yes']
OnlineBackup: ['Yes' 'No']
DeviceProtection: ['No' 'Yes']
TechSupport: ['No' 'Yes']
StreamingTV: ['No' 'Yes']
StreamingMovies: ['No' 'Yes']


### Verify final datatypes

In [12]:
print(df.dtypes)

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
numAdminTickets       int64
numTechTickets        int64
Churn                object
dtype: object


Let’s fix SeniorCitizen to make it cleaner for Power BI

In [13]:
df['SeniorCitizen'].unique()

array([0, 1])

In [14]:
df['SeniorCitizen'] = df['SeniorCitizen'].replace({1: 'Yes', 0: 'No'})
df['SeniorCitizen'].unique()

array(['No', 'Yes'], dtype=object)

In [15]:
df['TenureGroup'] = pd.cut(
    df['tenure'],
    bins=[0, 12, 24, 48, 60, df['tenure'].max()],
    labels=['0-12', '13-24', '25-48', '49-60', '61+'],
    include_lowest=True
)

In [16]:
df['TenureGroup'].isna().sum()

np.int64(0)

In [17]:
# Function to verify each row
def check_group(row):
    t = row['tenure']
    g = row['TenureGroup']
    if t <= 12 and g != '0-12': return False
    elif 13 <= t <= 24 and g != '13-24': return False
    elif 25 <= t <= 48 and g != '25-48': return False
    elif 49 <= t <= 60 and g != '49-60': return False
    elif t > 60 and g != '61+': return False
    return True

print(df.apply(check_group, axis=1).value_counts())

True    7043
Name: count, dtype: int64


In [18]:
df.groupby('TenureGroup')['tenure'].agg(['min', 'max', 'count'])

/var/folders/1y/fqrnl1wj2v32zp3hwv5ny6_r0000gn/T/ipykernel_97648/4208069606.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('TenureGroup')['tenure'].agg(['min', 'max', 'count'])


,min,max,count
TenureGroup,,,
0-12,0,12,2186
13-24,13,24,1024
25-48,25,48,1594
49-60,49,60,832
61+,61,72,1407


In [19]:
for col in df.columns:
    if col != 'customerID':
        print(f"{col}: {df[col].dtype} | {df[col].unique()}")

gender: object | ['Female' 'Male']
SeniorCitizen: object | ['No' 'Yes']
Partner: object | ['Yes' 'No']
Dependents: object | ['No' 'Yes']
tenure: int64 | [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
PhoneService: object | ['No' 'Yes']
MultipleLines: object | ['No' 'Yes']
InternetService: object | ['DSL' 'Fiber optic' 'No']
OnlineSecurity: object | ['No' 'Yes']
OnlineBackup: object | ['Yes' 'No']
DeviceProtection: object | ['No' 'Yes']
TechSupport: object | ['No' 'Yes']
StreamingTV: object | ['No' 'Yes']
StreamingMovies: object | ['No' 'Yes']
Contract: object | ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: object | ['Yes' 'No']
PaymentMethod: object | ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
MonthlyCharges: float64 | [29.85 56.95 53.85 ... 63.1  44.2

In [20]:
pd.crosstab(df['Contract'], df['Churn'])

Churn,No,Yes
Contract,,
Month-to-month,2220,1655
One year,1307,166
Two year,1647,48


✅ Now the data is ready for anlaysis, let's save it to a csv file

In [21]:
# Export clean CSV
df.to_csv('CustomerChurnCleaned.csv', index=False)